문서 요약 모델로 BART 채택  
KoBERT처럼 한국어로 사전 학습 시킨 KoBART가 있다고 들어서 가져다가 쓰는 것으로 했다.

[SKT-AI KoBART](https://github.com/SKT-AI/KoBART)  
처음에는 SKT-AI KoBART로 진행하려고 하였는데 의존성 설치하는 과정에서 내가 사용하고 있는 pytorch 2.0과 충돌이 나서 위 모델 사용은 포기하였다.  
그런데 찾아보니 hugginface에 [gogamza/kobart-base-v2](https://huggingface.co/gogamza/kobart-base-v2)가 동일한 pre-trained 모델인 것 같아 이를 사용해보았다.

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
model_name = 'cosmoquester/bart-ko-base'

# get model, tokenizer
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

데이터셋 : [문서요약 텍스트](https://www.aihub.or.kr/aihubdata/data/view.do?currMenu=&topMenu=&aihubDataSe=data&dataSetSn=97)  

위 데이터셋은 법률, 사설, 그리고 신문기사 총 세 가지 범주로 나뉘어 있는데, 우리의 관심 영역은 뉴스이기 때문에 뉴스에 대한 학습 데이터만 사용하도록 한다.  
기본적으로 제목, 본문, 요약 뿐 아니라 불용어 위치 정보, 미디어 명, 신뢰성과 같은 기사의 메타 데이터를 포함하고 있으므로 모듈 고도화 방향에서 적절한 데이터를 골라 사용하면 될 듯.  
일단 기본적인 요약 작업 테스트를 위해 본문의 내용과 요약된 라벨 데이터를 사용하여 학습을 진행하도록 한다.  
매번 학습을 시작할 때 마다 아래 전처리 작업을 진행하기에는 threading을 하여도 생각보다 오래 걸려서 생성된 객체를 .pt 파일로 저장하고 불러오는 방식으로 사용하려고 한다.  

In [ ]:
import json
from threading import Thread
from datasets import Dataset

creates_dataset = False
dataset_file = 'data/dataset.pt'
train_file = 'data/train_original.json' # num of total data is about 240000
valid_file = 'data/valid_original.json' # num of total data is about 30000
num_threads = 8

# read json & tokenize
def get_input_and_labels(documents, articles, abstractives):
    for document in documents:
        article = ''
        for text in document['text']:
            if len(text) > 0:
                article += (text[0]['sentence'] + ' ')
        articles.append(article)
        
        abstractive = document['abstractive']
        if len(abstractive) > 0:
            abstractive = abstractive[0]
        abstractives.append(abstractive)
        
def get_dataset_from_json(json_file, num_data=0):
    with open(json_file, 'r') as f:
        json_data = json.load(f)
        documents = json_data['documents']
        data_size = len(documents)
        if num_data == 0 or num_data > data_size:
            num_data = data_size
        
        data_per_threads = num_data//num_threads
        t_results = []
        threads = []
        for i in range(num_threads):
            t_result = [[], []]
            t_results.append(t_result)
            
            thread = Thread(target=get_input_and_labels, args=(documents[i*data_per_threads:(i+1)*data_per_threads], t_result[0], t_result[1],))
            thread.daemon = True
            thread.start()
            threads.append(thread)

        for thread in threads:
            thread.join()
        
        data_dict = {'article':[], 'abstractive':[]}
        for t_result in t_results:
            data_dict['article'].extend(t_result[0])
            data_dict['abstractive'].extend(t_result[1])
            
        return Dataset.from_dict(data_dict)

if creates_dataset:
    train_dataset = get_dataset_from_json(train_file)
    val_dataset = get_dataset_from_json(valid_file)

Batch 작업을 위해 모든 Input을 동일한 BART 최대 길이인 1024로 설정해 주었다.  
Input을 최대 길이로 padding 해주었기 때문에 계산 효율을 위해 attention_mask 역시 활용.  
Label은 loss 계산 당시 동일한 input과 동일한 길이가 아니면 오류가 발생해서 똑같이 1024로 설정 해주었다.  

In [ ]:
from torch.utils.data import DataLoader, TensorDataset

batch_size = 1

def preprocess(examples):
    inputs = tokenizer(examples['article'], return_tensors='pt', max_length=1024, padding='max_length', truncation=True)
    labels = tokenizer(examples['abstractive'], return_tensors='pt', max_length=1024, padding='max_length', truncation=True)
    inputs['labels'] = labels['input_ids']
    return inputs

def create_dataloader(dataset):
    input_ids = dataset['input_ids']
    attention_mask = dataset['attention_mask']
    labels = dataset['labels']
    tensor_dataset = TensorDataset(input_ids, attention_mask, labels)
    return DataLoader(tensor_dataset, batch_size=batch_size)

if creates_dataset:
    dataloader = {
        'train': create_dataloader(train_dataset.map(preprocess, batched=True).with_format("torch")),
        'val': create_dataloader(val_dataset.map(preprocess, batched=True).with_format("torch"))
    }
    torch.save(dataloader, dataset_file)
else:
    dataloader = torch.load(dataset_file)

학습된 모델은 아래와 같은 방식으로 huggingface에 올려두고 모듈 API형식으로 만들 때 내려 받는 방식으로 사용하면 될 듯!  
tokenizer는 한번만 올려두면 되고

In [ ]:
# model.push_to_hub('yeti-s/kobart-base-v2-news-summarization', use_auth_token=write_token)
# tokenizer.push_to_hub('yeti-s/kobart-base-v2-news-summarization', use_auth_token=write_token)

1 사이클 학습 하는데 하루가 걸려서 checkpoint를 만들 필요가 있다.  
학습에는 epoch마다 선형적으로 learning rate를 감소시키는 scheduler를 사용하였다.  
이번에 yolov8 보면서 느낀 것인데 1 사이클마다 validation 데이터로 평가를 진행하고 가장 높은 점수를 받은 데이터를 best로 따로 저장하는 것도 구현을 하면 좋을듯!

In [ ]:
# evaluate model
from tqdm import tqdm

@torch.no_grad()
def eval_model(model, val_dataloader):
    device = next(model.parameters()).device
    model.to(device)
    model.eval()
    total_loss = 0
    
    print('=== evaluate model')
    for _, data in enumerate(tqdm(val_dataloader)):
        data = [t.to(device) for t in data]
        inputs = {
            'input_ids': data[0],
            'attention_mask': data[1],
            'labels': data[2]
        }
        outputs =  model(**inputs)
        loss = outputs.loss
        total_loss += loss.item()
    
    total_loss /= len(val_dataloader)
    print(f'total loss : {total_loss}')
    
    return total_loss


In [ ]:
# train model
import os
from torch.optim import AdamW, lr_scheduler

class Checkpoint():
    def __init__(self, model, optimizer, scheduler) -> None:
        self.model = model
        self.optimizer = optimizer
        self.scheduler = scheduler
        self.epoch = 0
        self.last_step = -1
        self.best_loss = 1e20
        
    def set_root_dir(self, root_dir):
        if root_dir is not None:
            self.root_dir = root_dir
            self.path = os.path.join(root_dir, 'checkpoint.pt')
            
            if not os.path.exists(root_dir):
                os.makedirs(root_dir)
                
            if os.path.exists(self.path):
                self.load(self.path)
    
    def load(self, save_path):
        checkpoint = torch.load(save_path)
        self.model.load_state_dict(checkpoint['model'])
        self.optimizer.load_state_dict(checkpoint['optimizer'])
        self.scheduler.load_state_dict(checkpoint['scheduler'])
        self.epoch = checkpoint['epoch']
        self.last_step = checkpoint['last_step']
        self.best_loss = checkpoint['best_loss']
    
    def save(self):
        if not self.path is None:
            torch.save({
                'model' : self.model.state_dict(),
                'optimizer' : self.optimizer.state_dict(),
                'scheduler' : self.scheduler.state_dict(),
                'epoch' : self.epoch,
                'last_step' : self.last_step,
                'best_loss' : self.best_loss
            }, self.path)
        
    def step(self):
        self.optimizer.step()
        self.last_step += 1
    
    def eval(self, val_dataloader):
        if not self.root_dir is None:
            loss = eval_model(self.model, val_dataloader)
            if self.loss > loss:
                self.loss = loss
                torch.save(self.model.state_dict(), os.path.join(self.root_dir, 'best.pt'))
    
    def next(self):
        self.scheduler.step()
        self.epoch += 1
        self.last_step = -1
        self.save()
        
    def close(self):
        if not self.path is None and os.path.exists(self.path):
            os.remove(self.path)


def train_model(model, dataloader, checkpoint_dir=None, epochs=1, lr=2e-5, device=torch.device('cuda')):
    model.to(device)
    optimizer = AdamW(model.parameters(), lr=lr)
    scheduler = lr_scheduler.LambdaLR(optimizer=optimizer, lr_lambda=lambda epoch:0.95**epoch)
    checkpoint = Checkpoint(model, optimizer, scheduler)
    checkpoint.set_root_dir(checkpoint_dir)

    for epoch in range(checkpoint.epoch, epochs):
        print(f'=== train model {epoch}/{epochs}')
        model.train()
        num_trained = 0
        total_loss = 0
        
        for step, data in enumerate(tqdm(dataloader['train'])):
            if step <= checkpoint.last_step:
                continue
            
            data = [t.to(device) for t in data]
            inputs = {
                'input_ids': data[0],
                'attention_mask': data[1],
                'labels': data[2]
            }

            # get loss
            optimizer.zero_grad()
            outputs =  model(**inputs)
            loss = outputs.loss
            total_loss += loss.item()
            
            loss.backward()
            checkpoint.step()
            num_trained += 1
            
            # save checkpoint 
            if step % 1000 == 0:
                checkpoint.save()
                print(f'loss : {total_loss/num_trained}')
        
        checkpoint.eval(dataloader['val'])
        checkpoint.next()
        
    # remove checkpoint
    checkpoint.close()

In [ ]:
train_model(model, dataloader, './checkpoint', epochs=5)
torch.save(model, 'data/bart.pt')

아래는 학습 결과 테스트용

In [ ]:
# predict
@torch.no_grad()
def predict(model, sentence, max_length=128, device=torch.device('cuda')):
    input_ids = tokenizer(sentence, return_tensors='pt').to(device)
    gen_ids = model.generate(**input_ids, max_length=max_length, use_cache=True)
    generated = tokenizer.decode(gen_ids[0])
    return generated

In [ ]:
model.eval().cuda()
test_sentence = '이스라엘군(IDF)은 2일(현지시간) 팔레스타인 가자지구의 핵심 지역인 가자시티 포위를 완료하고 군사작전을 수행 중이라고 밝혔다고 현지 일간 하레츠가 보도했다. IDF 대변인 다니엘 하가리 소장은 이날 저녁 브리핑에서 "병력들이 하마스의 전초기지와 본부, 발사대, 기반시설 등을 공격하고 있으며 근접전에서 테러리스트들을 제거하고 있다"고 말했다. 하가리 소장은 또 이스라엘 북부 국경에서 진행 중인 레바논의 친이란 시아파 무장정파 헤즈볼라와의 교전과 관련해 "IDF는 말이 아닌 행동으로 대응할 것"이라고 강조했다. 이날 앞서 헤르지 할레지 IDF 참모총장은 공군기지에서 발표한 성명에서 가자시티를 포위 중이라고 밝히며 "우리는 전쟁에서 또 하나의 중요한 단계의 진전을 이뤘다"고 말했다고 일간 타임스오브이스라엘(TOI)이 보도했다. 그는 "병력은 밀집되고 복잡한 도시 지역에서 전투하고 있다"며 "정확한 정보와 공중과 바다에서의 공습 지원이 전투를 더욱 효과적으로 만들고 있다"고 설명했다. 할레비 참모총장은 지상전 개시 이후 지금까지 18명이 전사했다면서 "고통스러운 대가를 치르고 있지만, 우리는 계속해서 승리할 것"이라고 덧붙였다.'
print(predict(model, test_sentence))